# Exploration des meilleurs modèles

### Avec les explorations des modèles précédents, nous avons pu constater que :

De nombreux modèles présentent d'excellentes performances : Random Forest, Gradient Boosting, SVM, KNN, etc. Que ce soit pour la classification de la vitesse ou des déplacements, les résultats sont très satisfaisants.

L’idée initiale était d’explorer les hyperparamètres de ces modèles afin d’en améliorer les performances, mais au vu des résultats obtenus, cela ne semble pas nécessaire pour le moment.

Cependant, nous allons tout de même explorer les modèles de type ANN (Artificial Neural Networks), dans le but d’augmenter le taux de prédictions "neutres" pour les mouvements, et ainsi réduire les cas où des mouvements qui devraient être classés comme neutres ne le sont pas.

### ANN pour la classification des déplacements

##### Train split des données pour la classification des déplacements.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

data = pd.read_csv('../preprocessing_data/move_preprocess.csv')

X = data.drop('action', axis=1)
y = data['action']

le = LabelEncoder()
y = le.fit_transform(y)

print(le.classes_)


X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Nombre de colonne de X

X_train.shape[1]



##### Recherche des meilleurs hyperparamètres pour l'ANN avec RandomizedSearchCV

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from scikeras.wrappers import KerasClassifier
import numpy as np

def create_ann_model(input_dim, nb_outputs, nb_layers=2, nb_neurons=8, dropout_rate=0.2, activation='relu'):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    model.add(Dense(nb_neurons, activation=activation))
    model.add(Dropout(dropout_rate))
    for _ in range(nb_layers - 1):
        model.add(Dense(nb_neurons, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(nb_outputs, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

clf = KerasClassifier(
    model=create_ann_model,
    input_dim=X_train.shape[1],
    nb_outputs=len(np.unique(y_train)),
    verbose=0
)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

import numpy as np

param_dist = {
    'model__nb_layers': [2, 3, 5],
    'model__nb_neurons': [8, 16, 32, 64],
    'model__dropout_rate': [0.2, 0.3, 0.5],
    'model__activation': ['relu', 'tanh'],
    'epochs': [30, 50, 100],
    'batch_size': [32, 64, 128],
}

random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    verbose=1,
    scoring='accuracy',
    random_state=42
)

random_search.fit(X_train, y_train)

In [ ]:
print("Meilleurs hyperparamètres :")
print(random_search.best_params_)

print("Meilleur score (validation croisée) :")
print(random_search.best_score_)

from sklearn.metrics import accuracy_score

y_pred = random_search.predict(X_val)
print("Score sur validation :", accuracy_score(y_val, y_pred))

##### Recherche plus exhaustive des hyperparamètres pour l'ANN avec GridSearchCV

Une recherche plus exhaustive des hyperparamètres a été réalisée pour le modèle ANN, en utilisant GridSearchCV. Cette recherche s'est concentrée autour des meilleurs hyperparamètres identifiés précédemment par RandomizedSearchCV.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


clf = KerasClassifier(
    model=create_ann_model,
    input_dim=X_train.shape[1],
    nb_outputs=len(np.unique(y_train)),
    verbose=0
)

param_grid = {
    'model__nb_neurons': [16, 32],
    'model__nb_layers': [2, 3, 5],
    'model__activation': ['tanh'],
    'model__dropout_rate': [0.2],
    'epochs': [100],
    'batch_size': [32]
}

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

grid_search.fit(
    X_train, y_train,
    **{
        'callbacks': [early_stop],
        'validation_split': 0.2,
        'verbose': 1
    }
)


In [ ]:
print("Best params (Grid Search):", grid_search.best_params_)
print("Best score (Grid Search):", grid_search.best_score_)

##### Etude du modèle ANN avec les meilleurs hyperparamètres

Etude sans scikeras (entièrement avec keras)


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout


best_model = create_ann_model(input_dim=X_train.shape[1],
    nb_outputs=len(np.unique(y_train)),
    nb_layers=grid_search.best_params_['model__nb_layers'],
    nb_neurons=grid_search.best_params_['model__nb_neurons'],
    dropout_rate=grid_search.best_params_['model__dropout_rate'],
    activation=grid_search.best_params_['model__activation']
)
history = best_model.fit(
    X_train, y_train,
    epochs=grid_search.best_params_['epochs'],
    batch_size=grid_search.best_params_['batch_size'],
    validation_split=0.2,
    verbose=1
)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_prob = best_model.predict(X_val)
y_pred = y_pred_prob.argmax(axis=1)

print("Accuracy sur le jeu de validation :", accuracy_score(y_val, y_pred))

On constate une très bonne accuracy sur le jeu de validation, ce qui est prometteur pour la classification des déplacements. Les résultats ne sont pas étonnants, au vu des performances des modèles précédents.

In [ ]:
print(best_model.summary())

Etude de l'entrainement (erreur à chaque epoch) du modèle avec les meilleurs hyperparamètres trouvés.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Entraînement')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel("Épochs")
plt.ylabel("Erreur (loss)")
plt.title("Courbe d'apprentissage du meilleur modèle")
plt.legend()
plt.grid(True)
plt.show()

On constate que le modèle converge bien, avec une erreur de validation qui diminue au fil des époques. Il n'y a pas de sur-apprentissage évident, ce qui est un bon signe pour la généralisation du modèle. Malgré des meilleurs résultats étonnant sur le jeu de validation que sur le jeu d'entraînement.

##### Amélioration du modèle ANN grâce à l'étude de seuil

Maintenant le but va être de trouver un seuil idéale à partir des sorties des neurones pour augmenter le taux de prédictions neutres, et éviter ainsi des mouvements non souhaités (les minimiser en les classifiant comme neutres).

Pour cela nous allons étudier 2 méthodes différentes qui seront détaillés plus bas.

In [ ]:
print(le.classes_)

Neutre correspond à la classe 4

##### Les méthodes :

Méthode A avec delta comme seuil :
- pour chaque échantillon, on calcule la différence entre la probabilité maximale (p_max) et la seconde probabilité maximale (p_2nd) et on compare cette différence à un seuil delta. Si (p_max - p_2nd) < delta, on classe l'échantillon comme neutre.

Méthode B avec alpha comme seuil :
- pour chaque échantillon, on calcule la différence de la probabilité de la classe neutre (p_neutre) et la probabilité de la classe neutre et on la compare à un seuil alpha. Si p_neutre >= alpha, on classe l'échantillon comme neutre.

In [ ]:
# Code généré ici à l'aide chatGPT

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Méthode A
delta_list = [0.05, 0.10, 0.20]
# Méthode B
alpha_list = [0.30, 0.40, 0.50]

probas_train = best_model.predict(X_train)

print("Shape des probas_train :", probas_train.shape)

print("Accuracy sur le jeu d'entraînement :", accuracy_score(y_train, probas_train.argmax(axis=1)))

p_neutre_all = probas_train[:, 4]

# Pour calculer (p_max - p_2nd) pour chaque échantillon, on fait :
# 1) on trie chaque ligne de probas_train pour isoler p_max et p_2nd
# 2) on calcule la différence
sorted_indices = np.argsort(probas_train, axis=1)
# p_sorted[i, -1] = p_max pour l'échantillon i
# p_sorted[i, -2] = p_2nd pour l'échantillon i
p_sorted = np.take_along_axis(probas_train, sorted_indices, axis=1)
p_max_all = p_sorted[:, -1]    # vecteur de taille N_train
p_2nd_all = p_sorted[:, -2]    # vecteur de taille N_train

delta_all = p_max_all - p_2nd_all  # vecteur (p_max - p_2nd) pour chaque échantillon

# --- 2.4. Boucle sur toutes les combinaisons (delta, alpha) ---
# On va construire une liste de lignes pour un DataFrame, où chaque ligne contiendra :
#    [delta, alpha, accuracy]
records = []

for delta in delta_list:
    for alpha in alpha_list:
        # Pour chaque échantillon i, on décide la prédiction finale selon :
        #   si p_neutre >= alpha  OU  delta_i < delta  => classe 2 (neutre)
        #   sinon                                      => argmax(probas_train[i])
        # On peut vectoriser cette logique en deux étapes :

        # Étape 1 : on initialise pred = argmax(probas_train, axis=1)
        preds_argmax = np.argmax(probas_train, axis=1)

        # Étape 2 : on marque en 2 (neutre) tous les indices i où
        #           p_neutre_all[i] >= alpha   OU   delta_all[i] < delta
        # On crée un masque booléen "to_neutral" de taille N_train :
        to_neutral = np.logical_or(p_neutre_all >= alpha, delta_all < delta)

        # On remplace dans preds_argmax(i) par 2 si to_neutral[i] est True
        final_preds = preds_argmax.copy()
        final_preds[to_neutral] = 4

        # --- 2.5. Calcul de l’accuracy sur X_train/y_train pour ce (delta, alpha) ---
        acc = accuracy_score(y_train, final_preds)

        # On stocke le triplet
        records.append({
            'delta': delta,
            'alpha': alpha,
            'accuracy': acc
        })

# --- 2.6. Construction du DataFrame 3×3 « matrice » ---
df_results = pd.DataFrame(records)

# Pivot : ligne = delta, colonne = alpha, valeur = accuracy
matrix_acc = df_results.pivot(index='delta', columns='alpha', values='accuracy')

# Affichage
print("Matrice des accuracies (rows = delta, cols = alpha) :\n")
print(matrix_acc)


On remarque étonnament que les seuils améliorent l'accuracy du modèle, ce qui n'était pas attendu. En effet, on s'attendait à ce que les seuils réduisent l'accuracy en classifiant des échantillons comme neutres alors qu'ils ne le sont pas.

On conservera le seuil delta = 0.20 et alpha = 0.30, car il permet d'augmenter l'accuracy du modèle tout en réduisant les mouvements non souhaités. On les réutilisera dans model_API_ANN.py pour la classification des mouvements.

Sauvegarde du meilleur modèle qui sera utilisé dans l'API model_API_ANN.py, ainsi que du label encoder pour la classification des mouvements.

In [ ]:
# import joblib
#
# best_model.save('../models_ANN/best_ann_model_move.h5')
# joblib.dump(le, "../models_ANN/label_encoder_move.pkl")

Finalement je ne vais pas étudier la position 2 pour la vitesse de la même façon que le neutre du mouvement, cela ne servirait à rien car la position 2 n'est pas vraiment considéré comme une position neutre.